## CRM Agent with ADK

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Fcrm-agent-mcp.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/crm-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [92]:
%pip install google-adk -q
%pip install litellm -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [93]:
import os
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams

### Set Auth

In [ ]:
#os.environ["GOOGLE_API_KEY"] = "" # <--- REPLACE
CLOUD_PROJECT = "GCP_PROJECT_ID" # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"]= CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"]="us-central1"

### Get Apigee MCP Tools

In [ ]:
async def get_customer_tools_async():
  """Gets tools from the Customer MCP Server."""
  print("Attempting to connect to Customer MCP server...")
  
  mcp_base_url = "https://APIGEE_HOST/crm-mcp-proxy/sse"  # @param {type:"string"}
  if not mcp_base_url:
    raise ValueError("Variable MCP_BASE_URL is not set.")

  client_id = "CRM_TOOLS_API_KEY"  # @param {type:"string"}

  if not client_id:
    raise ValueError("Environment variable CRM_TOOLS_API_KEY is not set.")

  tools, exit_stack = await MCPToolset.from_server(
      connection_params=SseServerParams(
          url=mcp_base_url,
          headers={"x-api-key": client_id},
      )
  )
  print("MCP Customer Toolset created successfully.")
  return tools, exit_stack

### Initialize Agent

In [96]:
async def get_customer_agent_async():
    """Creates a Customer Agent"""
    tools, exit_stack = await get_customer_tools_async()
    print(f"Fetched {len(tools)} customer tools from MCP server.")
    customer_agent = Agent(
        model="gemini-2.5-pro-preview-05-06",
        name="customer_agent",
        description="Central authority for all customer account data and related engagement triggers",
        instruction="""
        - You are responsible for managing customer account data. Your primary function is to use the available tool operations to accurately and securely handle customer information.
          - **Manage user profiles:**
            - Retrieve comprehensive profile details for a specific customer using their customer ID. This can be used for purposes like pre-filling order forms.
          - **Manage customer addresses:**
            - Retrieve a list of all saved addresses for a specific customer.
            - Retrieve the details of a specific address for a customer.
        """,
        tools=tools
    )
    return customer_agent, exit_stack

### Build Agent Runtime

In [97]:
async def async_main():
  session_service = InMemorySessionService()
  artifacts_service = InMemoryArtifactService()

  session = session_service.create_session(
      state={}, app_name='crm_app', user_id='user_fs'
  )

  query = "get details about customer 1234"
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent, exit_stack = await get_customer_agent_async()

  runner = Runner(
      app_name='crm_app',
      agent=root_agent,
      artifact_service=artifacts_service,
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=session.user_id, new_message=content
  )

  async for event in events_async:
    print(f"Event received: {event}")

  # Crucial Cleanup: Ensure the MCP server process connection is closed.
  print("Closing MCP server connection...")
  await exit_stack.aclose()
  print("Cleanup complete.")

### Execute Agent

In [ ]:
await async_main()